In [59]:
import pandas as pd
import ast

# 1. Load the CSV (with mixed-type handling)
file_path = '../../all_materials_thermo.csv'

try:
    df = pd.read_csv(
        file_path,
        low_memory=False,
        na_values=['NA', 'N/A', '--', '-', ''],
        encoding='utf-8'
    )
    print("✅ File loaded successfully!")
except Exception as e:
    print(f"❌ Error loading file: {e}")
    exit()

# 2. Helper function: Safely parse a tuple from string
def try_parse_tuple(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

# 3. Step: Parse each cell into a tuple if possible
for col in df.columns:
    df[col] = df[col].apply(try_parse_tuple)

# 4. Step: Create new column names from the first element of the tuple
new_columns = {}
for col in df.columns:
    first_value = df[col].dropna().iloc[0]
    if isinstance(first_value, tuple) and len(first_value) == 2:
        new_col_name = first_value[0]  # First part of the tuple
    else:
        new_col_name = col
    new_columns[col] = new_col_name

df = df.rename(columns=new_columns)

# 5. Step: Replace each cell with just the second value
for col in df.columns:
    df[col] = df[col].apply(lambda x: x[1] if isinstance(x, tuple) and len(x) == 2 else x)
    
output_path = 'cleaned_thermo_materials.csv'
df.to_csv(output_path, index=False)
print(f"\n💾 Cleaned data saved to '{output_path}'")

✅ File loaded successfully!

💾 Cleaned data saved to 'cleaned_thermo_materials.csv'


In [60]:
df.head(2)

,0,nsites,2,nelements,4,5,formula_pretty,formula_anonymous,chemsys,volume,...,energy_above_hull,is_stable,equilibrium_reaction_energy_per_atom,decomposes_to,decomposition_enthalpy,31,energy_type,entry_types,34,fields_not_requested
0,"('builder_meta', EmmetMeta(emmet_version='0.84...",8,"('elements', [Element O])",1,"('composition', Composition('O8'))","('composition_reduced', Composition('O2'))",O2,A,O,107.643808,...,0.0,True,0.0,None,0.0,"('decomposition_enthalpy_decomposes_to', [Deco...",GGA,[GGA],"('entries', {'GGA': mp-12957-GGA ComputedStruc...",[]
1,"('builder_meta', EmmetMeta(emmet_version='0.84...",2,"('elements', [Element Ce])",1,"('composition', Composition('Ce2'))","('composition_reduced', Composition('Ce1'))",Ce,A,Ce,52.180213,...,0.0,True,0.0,None,0.0,"('decomposition_enthalpy_decomposes_to', [Deco...",GGA,[GGA],"('entries', {'GGA': mp-567332-GGA ComputedStru...",[]


In [61]:
# drop unnecessary columns
columns_to_keep = [3, 4, 9, 10,14, 22, 23, 24, 25, 26, 28]

if max(columns_to_keep) < len(df.columns):
    df = df.iloc[:, columns_to_keep]
else:
    print("❌ One of the column indices exceeds the number of columns in the DataFrame.")

In [62]:
# drop rows with less then two elements
df = df[df.iloc[:, 0] >= 2]

# check data structure
df.head(2)

,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,energy_uncertainy_per_atom,formation_energy_per_atom,energy_above_hull,equilibrium_reaction_energy_per_atom
88,2,"('composition', Composition('H6 F4'))",107.897600,1.262611,"('material_id', MPID(mp-1184796))",-4.024832,-4.209632,None,-1.409404,0.0,-0.028876
89,2,"('composition', Composition('Zn35 Cu17'))",703.898146,7.949087,"('material_id', MPID(mp-1216020))",-2.289258,-2.289258,None,-0.101228,0.0,-0.004592


In [63]:
# save to csv

output_path = 'cleaned_thermo_materials.csv'
df.to_csv(output_path, index=False)
print(f"\n💾 Cleaned data saved to '{output_path}'")


💾 Cleaned data saved to 'cleaned_thermo_materials.csv'


In [64]:
import re

def extract_elements(comp_str):
    if pd.isna(comp_str):
        return []
    
    # Match the formula part inside Composition('...')
    match = re.search(r"Composition\('([^']+)'\)", comp_str)
    if not match:
        return []
    
    formula = match.group(1)
    # Extract element symbols using regex (e.g., "H6 F4" → ["H", "F"])
    elements = re.findall(r'[A-Z][a-z]?', formula)
    return list(set(elements))  # Optional: remove duplicates

# Apply to column 2
df['elements'] = df.iloc[:, 1].apply(extract_elements)

# check data structure
df.head(2)

,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,energy_uncertainy_per_atom,formation_energy_per_atom,energy_above_hull,equilibrium_reaction_energy_per_atom,elements
88,2,"('composition', Composition('H6 F4'))",107.897600,1.262611,"('material_id', MPID(mp-1184796))",-4.024832,-4.209632,None,-1.409404,0.0,-0.028876,"[H, F]"
89,2,"('composition', Composition('Zn35 Cu17'))",703.898146,7.949087,"('material_id', MPID(mp-1216020))",-2.289258,-2.289258,None,-0.101228,0.0,-0.004592,"[Cu, Zn]"


In [65]:
# Extract the formula from the composition string
def extract_formula(comp_str):
    match = re.search(r"Composition\('([^']+)'\)", comp_str)
    return match.group(1) if match else None

# Apply to column 2 (index 1)
df.iloc[:, 1] = df.iloc[:, 1].apply(extract_formula)

# check data structure
df.head(2)

,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,energy_uncertainy_per_atom,formation_energy_per_atom,energy_above_hull,equilibrium_reaction_energy_per_atom,elements
88,2,H6 F4,107.897600,1.262611,"('material_id', MPID(mp-1184796))",-4.024832,-4.209632,None,-1.409404,0.0,-0.028876,"[H, F]"
89,2,Zn35 Cu17,703.898146,7.949087,"('material_id', MPID(mp-1216020))",-2.289258,-2.289258,None,-0.101228,0.0,-0.004592,"[Cu, Zn]"


In [66]:
CERAMIC_ELEMENTS = [
    "Si", "Al", "Mg", "Zr", "Ti", "Ca", "Y", "Hf",
    "Fe", "Na", "K", "Ba", "Sr", "Li", "Be", "Mn", "V", "Cr", 
    "Nb", "Mo", "W", "Re", "Sc", "La", "Ce", "Th", "U"
]

ALL_CERAMIC_ELEMENTS = [
    "O", "N", "C", "B", "Si", "Al", "Mg", "Zr", "Ti", "Ca", "Y", "Hf",
    "Fe", "Na", "K", "Ba", "Sr", "Li", "Be", "Mn", "V", "Cr", 
    "Nb", "Mo", "W", "Re", "Sc", "La", "Ce", "Th", "U",
]

# Basic classification (simplified)
NON_METALS = {"O", "N", "C", "B"}
METALS = set(CERAMIC_ELEMENTS) - NON_METALS

# Check if composition has at least one metal and one non-metal
def has_metal_and_nonmetal(elements):
    return any(e in METALS for e in elements) and any(e in NON_METALS for e in elements)

def all_elements_valid(elements):
    return all(e in ALL_CERAMIC_ELEMENTS for e in elements)

# Apply to column 12
df = df[df.iloc[:, 11].apply(all_elements_valid)]
df = df[df.iloc[:, 11].apply(has_metal_and_nonmetal)]

# Save to CSV
df.to_csv("filtered_cleaned_thermo_materials.csv", index=False)
print("✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.")

✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.


In [67]:
# show data structure
df.head(2)

,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,energy_uncertainy_per_atom,formation_energy_per_atom,energy_above_hull,equilibrium_reaction_energy_per_atom,elements
109,2,U16 N28,600.250060,11.620730,"('material_id', MPID(mp-32590))",-10.839436,-11.069164,None,-1.658154,0.0,-0.081170,"[U, N]"
205,2,Ti6 O10,177.344754,4.187248,"('material_id', MPID(mp-1147))",-9.003853,-9.433228,None,-3.379943,0.0,-0.002179,"[Ti, O]"


In [68]:
# Extract material ID from the string
def extract_material_id(id_str):
    match = re.search(r"MPID\(([^)]+)\)", id_str)
    return match.group(1) if match else None

# Apply to column 5 (index 4)
df.iloc[:, 4] = df.iloc[:, 4].apply(extract_material_id)

# Save to CSV
df.to_csv("filtered_cleaned_thermo_materials.csv", index=False)
print("✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.")

# show data structure
df.head(2)

✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.


,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,energy_uncertainy_per_atom,formation_energy_per_atom,energy_above_hull,equilibrium_reaction_energy_per_atom,elements
109,2,U16 N28,600.250060,11.620730,mp-32590,-10.839436,-11.069164,None,-1.658154,0.0,-0.081170,"[U, N]"
205,2,Ti6 O10,177.344754,4.187248,mp-1147,-9.003853,-9.433228,None,-3.379943,0.0,-0.002179,"[Ti, O]"


In [69]:
# drop unnecessary columns and save
df.drop(df.columns[[7, 9]], axis=1, inplace=True)

# Save to CSV
df.to_csv("filtered_cleaned_thermo_materials.csv", index=False)
print("✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.")

# show data structure
df.head(2)

✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.


,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,formation_energy_per_atom,equilibrium_reaction_energy_per_atom,elements
109,2,U16 N28,600.250060,11.620730,mp-32590,-10.839436,-11.069164,-1.658154,-0.081170,"[U, N]"
205,2,Ti6 O10,177.344754,4.187248,mp-1147,-9.003853,-9.433228,-3.379943,-0.002179,"[Ti, O]"


In [70]:
# drop unnecessary columns and save
df.drop(df.columns[[9]], axis=1, inplace=True)

# Save to CSV
df.to_csv("filtered_cleaned_thermo_materials.csv", index=False)
print("✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.")

# show data structure
df.head(2)

✅ Saved as 'filtered_cleaned_thermo_materials.csv' with only ceramic compositions.


,nelements,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,formation_energy_per_atom,equilibrium_reaction_energy_per_atom
109,2,U16 N28,600.250060,11.620730,mp-32590,-10.839436,-11.069164,-1.658154,-0.081170
205,2,Ti6 O10,177.344754,4.187248,mp-1147,-9.003853,-9.433228,-3.379943,-0.002179


In [71]:
# drop unnecessary columns and save
df.drop(df.columns[[0]], axis=1, inplace=True)

# Save to CSV
df.to_csv("filtered_cleaned_thermo_materials.csv", index=False)
print("✅ Saved as 'filtered_cleaned_thermo_materials.csv'.")

# show data structure
df.head(2)

✅ Saved as 'filtered_cleaned_thermo_materials.csv'.


,4,volume,density,14,uncorrected_energy_per_atom,energy_per_atom,formation_energy_per_atom,equilibrium_reaction_energy_per_atom
109,U16 N28,600.250060,11.620730,mp-32590,-10.839436,-11.069164,-1.658154,-0.081170
205,Ti6 O10,177.344754,4.187248,mp-1147,-9.003853,-9.433228,-3.379943,-0.002179


In [72]:
# update column names
df.columns.values[0] = "Formula"
df.columns.values[1] = "Volume"
df.columns.values[2] = "Density"
df.columns.values[3] = "IDs"
df.columns.values[4] = "Uncorrected EPA"
df.columns.values[5] = "EPA"
df.columns.values[6] = "Formation EPA"
df.columns.values[7] = "Equilibrium Reaction EPA"

# Save to CSV
df.to_csv("filtered_cleaned_thermo_materials.csv", index=False)
print("✅ Saved as 'filtered_cleaned_thermo_materials.csv'.")

# show data structure
df.head(2)

✅ Saved as 'filtered_cleaned_thermo_materials.csv'.


,Formula,Volume,Density,IDs,Uncorrected EPA,EPA,Formation EPA,Equilibrium Reaction EPA
109,U16 N28,600.250060,11.620730,mp-32590,-10.839436,-11.069164,-1.658154,-0.081170
205,Ti6 O10,177.344754,4.187248,mp-1147,-9.003853,-9.433228,-3.379943,-0.002179
